<a href="https://colab.research.google.com/github/amanullah20/rdd_project/blob/main/ml_using_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=76f747aba5e4bef63cc74cc1e03aa787da1ac002cd3f2b65aa9afbbc88152649
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
spark= SparkSession.builder.getOrCreate()

In [4]:
dataframe=spark.read.csv('/content/Admission_Prediction.csv',header=True,inferSchema=True)

In [5]:
type(dataframe)

pyspark.sql.dataframe.DataFrame

In [6]:
dataframe.show(5)

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|    337.0|      118.0|                4|4.5|4.5|9.65|     1.0|           0.92|
|    324.0|      107.0|                4|4.0|4.5|8.87|     1.0|           0.76|
|     NULL|      104.0|                3|3.0|3.5| 8.0|     1.0|           0.72|
|    322.0|      110.0|                3|3.5|2.5|8.67|     1.0|            0.8|
|    314.0|      103.0|                2|2.0|3.0|8.21|     0.0|           0.65|
+---------+-----------+-----------------+---+---+----+--------+---------------+
only showing top 5 rows



In [7]:
dataframe.printSchema()

root
 |-- GRE Score: double (nullable = true)
 |-- TOEFL Score: double (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: double (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [8]:
dataframe.columns

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit']

In [9]:
from pyspark.sql.functions import col , count, isnan, when
for c in dataframe.columns:
  print(col(c))

Column<'GRE Score'>
Column<'TOEFL Score'>
Column<'University Rating'>
Column<'SOP'>
Column<'LOR'>
Column<'CGPA'>
Column<'Research'>
Column<'Chance of Admit'>


In [10]:
dataframe.select(*(col(c) for c in dataframe.columns)).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|    337.0|      118.0|                4|4.5|4.5|9.65|     1.0|           0.92|
|    324.0|      107.0|                4|4.0|4.5|8.87|     1.0|           0.76|
|     NULL|      104.0|                3|3.0|3.5| 8.0|     1.0|           0.72|
|    322.0|      110.0|                3|3.5|2.5|8.67|     1.0|            0.8|
|    314.0|      103.0|                2|2.0|3.0|8.21|     0.0|           0.65|
|    330.0|      115.0|                5|4.5|3.0|9.34|     1.0|            0.9|
|    321.0|      109.0|             NULL|3.0|4.0| 8.2|     1.0|           0.75|
|    308.0|      101.0|                2|3.0|4.0| 7.9|     0.0|           0.68|
|    302.0|      102.0|                1|2.0|1.5| 8.0|     0.0|            0.5|
|    323.0|      108.0|                3

In [11]:
dataframe.select(*(col(c).cast('float') for c in dataframe.columns)).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|
|     NULL|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|
|    321.0|      109.0|             NULL|3.0|4.0| 8.2|     1.0|           0.75|
|    308.0|      101.0|              2.0|3.0|4.0| 7.9|     0.0|           0.68|
|    302.0|      102.0|              1.0|2.0|1.5| 8.0|     0.0|            0.5|
|    323.0|      108.0|              3.0

In [12]:
new_dataframe= dataframe.select(*(col(c).cast('float') for c in dataframe.columns))

In [13]:
new_dataframe.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



# **checking for null values in columns**

In [14]:
 new_dataframe.select([count(when(col(c).isNull(),c)).alias(c) for c in new_dataframe.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [15]:
new_dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in new_dataframe.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



# **For removing null values**

In [16]:
from pyspark.ml.feature import Imputer

In [17]:
imputer= Imputer(inputCols=['GRE Score','TOEFL Score','University Rating'],
                 outputCols=['GRE Score','TOEFL Score','University Rating'])
model = imputer.fit(new_dataframe)
imputed_data=model.transform(new_dataframe)


In [18]:
imputed_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|
|316.55878|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|
|    321.0|      109.0|        3.1216495|3.0|4.0| 8.2|     1.0|           0.75|
|    308.0|      101.0|              2.0|3.0|4.0| 7.9|     0.0|           0.68|
|    302.0|      102.0|              1.0|2.0|1.5| 8.0|     0.0|            0.5|
|    323.0|      108.0|              3.0

In [19]:
imputed_data.select([count(when(col(c).isNull(),c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [20]:
features = imputed_data.drop('Chance of Admit')

In [21]:
features.columns

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research']

In [22]:
assembler=VectorAssembler(inputCols=features.columns,outputCol='features')

In [23]:
output = assembler.transform(imputed_data)

In [24]:
output.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|            features|
+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|[337.0,118.0,4.0,...|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|[324.0,107.0,4.0,...|
|316.55878|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|[316.558776855468...|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|[322.0,110.0,3.0,...|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|[314.0,103.0,2.0,...|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|[330.0,115.0,5.0,...|
|    321.0|      109.0|        3.1216495|3.0|4.0| 8.2|     1.0|           0.75|[321.0,109.0

In [25]:
output.select('features').toPandas().values

array([[DenseVector([337.0, 118.0, 4.0, 4.5, 4.5, 9.65, 1.0])],
       [DenseVector([324.0, 107.0, 4.0, 4.0, 4.5, 8.87, 1.0])],
       [DenseVector([316.5588, 104.0, 3.0, 3.0, 3.5, 8.0, 1.0])],
       [DenseVector([322.0, 110.0, 3.0, 3.5, 2.5, 8.67, 1.0])],
       [DenseVector([314.0, 103.0, 2.0, 2.0, 3.0, 8.21, 0.0])],
       [DenseVector([330.0, 115.0, 5.0, 4.5, 3.0, 9.34, 1.0])],
       [DenseVector([321.0, 109.0, 3.1216, 3.0, 4.0, 8.2, 1.0])],
       [DenseVector([308.0, 101.0, 2.0, 3.0, 4.0, 7.9, 0.0])],
       [DenseVector([302.0, 102.0, 1.0, 2.0, 1.5, 8.0, 0.0])],
       [DenseVector([323.0, 108.0, 3.0, 3.5, 3.0, 8.6, 0.0])],
       [DenseVector([325.0, 106.0, 3.0, 3.5, 4.0, 8.4, 1.0])],
       [DenseVector([327.0, 111.0, 4.0, 4.0, 4.5, 9.0, 1.0])],
       [DenseVector([316.5588, 112.0, 4.0, 4.0, 4.5, 9.1, 1.0])],
       [DenseVector([307.0, 109.0, 3.0, 4.0, 3.0, 8.0, 1.0])],
       [DenseVector([311.0, 104.0, 3.0, 3.5, 2.0, 8.2, 1.0])],
       [DenseVector([314.0, 105.0, 3.0, 3

In [26]:
data = output.select('features','Chance of Admit')

In [27]:
data.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[337.0,118.0,4.0,...|           0.92|
|[324.0,107.0,4.0,...|           0.76|
|[316.558776855468...|           0.72|
|[322.0,110.0,3.0,...|            0.8|
|[314.0,103.0,2.0,...|           0.65|
|[330.0,115.0,5.0,...|            0.9|
|[321.0,109.0,3.12...|           0.75|
|[308.0,101.0,2.0,...|           0.68|
|[302.0,102.0,1.0,...|            0.5|
|[323.0,108.0,3.0,...|           0.45|
|[325.0,106.0,3.0,...|           0.52|
|[327.0,111.0,4.0,...|           0.84|
|[316.558776855468...|           0.78|
|[307.0,109.0,3.0,...|           0.62|
|[311.0,104.0,3.0,...|           0.61|
|[314.0,105.0,3.0,...|           0.54|
|[317.0,107.0,3.0,...|           0.66|
|[319.0,106.0,3.0,...|           0.65|
|[318.0,110.0,3.0,...|           0.63|
|[303.0,102.0,3.0,...|           0.62|
+--------------------+---------------+
only showing top 20 rows



In [28]:
train_df,test_df = data.randomSplit([0.7,0.3])

In [29]:
test_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,2.0,2...|           0.57|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,101.0,1.0,...|            0.6|
|[298.0,105.0,3.0,...|           0.69|
|[299.0,94.0,1.0,1...|           0.42|
|[300.0,97.0,2.0,3...|           0.65|
|[300.0,102.0,2.0,...|           0.56|
|[300.0,105.0,1.0,...|           0.58|
|[301.0,96.0,1.0,3...|           0.54|
|[301.0,97.0,2.0,3...|           0.44|
|[301.0,99.0,2.0,3...|           0.64|
|[302.0,99.0,2.0,1...|           0.56|
|[302.0,101.0,2.0,...|           0.46|
|[302.0,102.0,3.0,...|           0.65|
|[303.0,98.0,1.0,2...|           0.56|
|[303.0,105.0,5.0,...|           0.77|
|[304.0,103.0,5.0,...|           0.71|
|[305.0,102.0,2.0,...|           0.62|
+--------------------+---------------+
only showing top 20 rows



# **Linear Regressor**

In [30]:
lin_reg=LinearRegression(featuresCol='features',labelCol='Chance of Admit')
linear_model=lin_reg.fit(train_df)

In [31]:
print('Coefficient :',linear_model.coefficients)
print('Intercept :', linear_model.intercept )

Coefficient : [0.0020494714043539325,0.0022668749192128247,0.0013738068482112846,0.00430774127736965,0.014057069098285368,0.12416613223020082,0.029024856016362878]
Intercept : -1.3191530083897658


In [32]:
trainSummary=linear_model.summary
print('RMSE :' , trainSummary.rootMeanSquaredError)
print('r2 Score :', trainSummary.r2)

RMSE : 0.06099908256111798
r2 Score : 0.8156071573178785


# **prediction**

In [33]:
predictions= linear_model.transform(test_df)
predictions.select('prediction','Chance of Admit','features').show()

+-------------------+---------------+--------------------+
|         prediction|Chance of Admit|            features|
+-------------------+---------------+--------------------+
|0.45176384146404547|           0.47|[295.0,96.0,2.0,1...|
|  0.515420770483858|           0.57|[295.0,99.0,2.0,2...|
|  0.511866108644834|           0.44|[296.0,95.0,2.0,3...|
| 0.5131966133500667|           0.49|[296.0,97.0,2.0,1...|
| 0.5243551356893981|            0.6|[296.0,101.0,1.0,...|
| 0.6654168925429746|           0.69|[298.0,105.0,3.0,...|
|0.43784323065785413|           0.42|[299.0,94.0,1.0,1...|
| 0.6081878993500556|           0.65|[300.0,97.0,2.0,3...|
| 0.5414204649271641|           0.56|[300.0,102.0,2.0,...|
| 0.5360018208342887|           0.58|[300.0,105.0,1.0,...|
| 0.5245791361941359|           0.54|[301.0,96.0,1.0,3...|
| 0.5829207885078311|           0.44|[301.0,97.0,2.0,3...|
| 0.5865891171361248|           0.64|[301.0,99.0,2.0,3...|
| 0.5489815137211635|           0.56|[302.0,99.0,2.0,1..

# **Evaluation**

In [34]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator= RegressionEvaluator(predictionCol='prediction',labelCol='Chance of Admit',metricName='r2')
print('R Squared (R2) on text data = ',pred_evaluator.evaluate(predictions))

R Squared (R2) on text data =  0.8289929079193163


# **Random Forest Regressor**

In [35]:
random_forest_reg= RandomForestRegressor(featuresCol="features",labelCol="Chance of Admit")

In [36]:
model = random_forest_reg.fit(train_df)

In [37]:
predictions= model.transform(test_df)

In [38]:
predictions.show()

+--------------------+---------------+-------------------+
|            features|Chance of Admit|         prediction|
+--------------------+---------------+-------------------+
|[295.0,96.0,2.0,1...|           0.47|0.46523733902151754|
|[295.0,99.0,2.0,2...|           0.57|  0.499979978346006|
|[296.0,95.0,2.0,3...|           0.44|0.48408351522640924|
|[296.0,97.0,2.0,1...|           0.49| 0.5317530755549253|
|[296.0,101.0,1.0,...|            0.6| 0.5239262881715895|
|[298.0,105.0,3.0,...|           0.69| 0.6347501989756636|
|[299.0,94.0,1.0,1...|           0.42|0.47074443394566057|
|[300.0,97.0,2.0,3...|           0.65| 0.6265720069855261|
|[300.0,102.0,2.0,...|           0.56| 0.5459914736183695|
|[300.0,105.0,1.0,...|           0.58| 0.5396973561010385|
|[301.0,96.0,1.0,3...|           0.54| 0.5220832871602145|
|[301.0,97.0,2.0,3...|           0.44| 0.5943773256102972|
|[301.0,99.0,2.0,3...|           0.64| 0.6148399808467948|
|[302.0,99.0,2.0,1...|           0.56| 0.543124148087875

# **Evaluation**

In [39]:
evaluator = RegressionEvaluator(labelCol='Chance of Admit',predictionCol='prediction',metricName='rmse')
print('Root Mean Squared Error (RMSE) on test data =',evaluator.evaluate(predictions))

Root Mean Squared Error (RMSE) on test data = 0.061147681421266226


In [40]:
evaluator= RegressionEvaluator(labelCol='Chance of Admit', predictionCol='prediction',metricName='r2')
print('R Squared (R2) on test data = ' , evaluator.evaluate(predictions))

R Squared (R2) on test data =  0.8032062743826195
